In [13]:
! pip install dash

In [14]:
from dash import Dash, dcc, html, Input, Output, State, callback_context
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import MarkerCluster
from io import BytesIO
import re
import numpy as np
import json
from dash.dependencies import ALL

In [16]:
df = pd.read_csv('dataset_cleaned.csv')

# For themes
light_theme = {
    'bg': '#f9f9f9',
    'card': 'white',
    'text': '#111111',
    'plot_bg': '#f0f0f0',
    'paper_bg': 'white'
}

dark_theme = {
    'bg': '#1e1e1e',
    'card': '#2c2c2c',
    'text': 'white',
    'plot_bg': '#333333',
    'paper_bg': '#2c2c2c'
}

# Ensure datetime columns are properly formatted
df['Date_datetime'] = pd.to_datetime(df['Date_datetime'], errors='coerce')
df['Time_datetime'] = pd.to_datetime(df['Time'], format='%I:%M %p', errors='coerce')
df['Hour'] = df['Time_datetime'].dt.hour

# Add Month_Name column if not exists
if 'Month_Name' not in df.columns:
    df['Month_Name'] = df['Date_datetime'].dt.month_name()

# Create time period classifications
def classify_time_period(hour):
    if pd.isna(hour):
        return 'All Day'  # N/A values show in All Day
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour <= 23 or 0 <= hour < 6:
        return 'Evening'
    else:
        return 'All Day'

df['Time_Period'] = df['Hour'].apply(classify_time_period)

def clean_vehicle_name(name):
    if not isinstance(name, str) or pd.isna(name):
        return None
    name = name.strip().upper()
    name = re.sub(r"[^A-Z0-9\s]", "", name)
    name = re.sub(r"\b\d+\s*", "", name)

    vehicle_map = {
        'BU': 'BUS',
        'MC': 'MOTORCYCLE',
        'PUJ': 'JEEP',
        'AUV': 'CAR',
        'SUV': 'CAR'
    }
    if name in vehicle_map:
        name = vehicle_map[name]

    if name.endswith('S') and name != 'BUS':
        name = name[:-1]

    return name

# Process vehicles for bar chart
df['Involved'] = df['Involved'].astype(str)
df['Involved'] = df['Involved'].str.upper().str.replace(r"[^A-Z0-9\s,]", "", regex=True)

# Helper function to create "No data" figure
def create_no_data_figure(title, theme_colors, message="No data available for the selected filters"):
    fig = go.Figure()
    fig.add_annotation(
        x=0.5, y=0.5,
        xref="paper", yref="paper",
        text=message,
        showarrow=False,
        font=dict(size=16, color=theme_colors['text']),
        bgcolor=theme_colors['plot_bg'],
        bordercolor=theme_colors['text'],
        borderwidth=1,
        borderpad=20
    )
    fig.update_layout(
        title=title,
        xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
        yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
        plot_bgcolor=theme_colors['plot_bg'],
        paper_bgcolor=theme_colors['paper_bg'],
        font_color=theme_colors['text'],
        height=400
    )
    return fig

app = Dash(__name__)
app.title = "WatchMetro: Metro Manila Accidents Dashboard"

# Filter Options
time_buttons = [
    {'label': 'All Day', 'value': 'All Day'},
    {'label': 'Morning', 'value': 'Morning'},
    {'label': 'Afternoon', 'value': 'Afternoon'},
    {'label': 'Evening', 'value': 'Evening'}
]

month_options = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

# Get unique cities from the dataset for city filter
city_options = sorted(df['City'].dropna().unique().tolist())

# Generate initial map
def generate_folium_map(data, theme):
    if data.empty:
        m = folium.Map(location=[14.6, 121.0], zoom_start=11)
        folium.Marker([14.6, 121.0], popup="No data for selected filters").add_to(m)
        return m

    center_lat = data['Latitude'].mean()
    center_lon = data['Longitude'].mean()

    # Use different tiles based on theme (though the dark theme doesn't look that good)
    tiles = 'CartoDB dark_matter' if theme == 'dark' else 'OpenStreetMap'
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles=tiles)

    cluster = MarkerCluster().add_to(m)
    for _, row in data.iterrows():
        popup = f"""
        <b>City:</b> {row['City']}<br>
        <b>Time:</b> {row['Time']}<br>
        <b>Date:</b> {row['Date']}<br>
        <b>Location:</b> {row['Location']}<br>
        <b>Involved:</b> {row['Involved']}<br>
        <b>Type:</b> {row['Type']}
        """
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup,
            icon=folium.Icon(color='red', icon='exclamation-triangle', prefix='fa')
        ).add_to(cluster)
    return m

# Helper function to get time range for filtering hourly data
def get_time_range(time_period):
    if time_period == 'Morning':
        return list(range(6, 12))
    elif time_period == 'Afternoon':
        return list(range(12, 18))
    elif time_period == 'Evening':
        return list(range(18, 24)) + list(range(0, 6))
    else:  # All Day
        return list(range(24))

# Create initial map
initial_data = df.dropna(subset=['Latitude', 'Longitude'])
initial_map = generate_folium_map(initial_data, 'light')
buffer = BytesIO()
initial_map.save(buffer, close_file=False)
initial_map_html = buffer.getvalue().decode("utf-8")

# App Layout
app.layout = html.Div([
    # Header with theme toggle in upper right
    html.Div([
        html.Div([
            html.H1("WatchMetro: A Dashboard of Metro Manila Accidents",
                    id='title', className='title',
                    style={'margin': '0'}),
            html.H3("Visualizing Traffic Incidents Across Metro Manila",
                    id='subtitle', className='subtitle',
                    style={'margin': '5px 0 0 0', 'color': '#666'})
        ], style={'flex': '1'}),

        html.Div([
            html.Div([
                html.Span("☀️", id='light-icon', className='theme-icon'),
                html.Div([
                    dcc.Checklist(
                        id='theme-toggle',
                        options=[{'label': '', 'value': 'dark'}],
                        value=[],
                        className='toggle-checklist',
                        inputStyle={'position': 'absolute', 'opacity': 0, 'width': 0, 'height': 0},
                        labelStyle={'display': 'block', 'width': '50px', 'height': '24px', 'position': 'relative', 'cursor': 'pointer'}
                    )
                ], className='toggle-switch'),
                html.Span("🌙", id='dark-icon', className='theme-icon')
            ], className='theme-toggle-container')
        ], style={'display': 'flex', 'alignItems': 'center'})
    ], className='header-container'),

    # Control Row
    html.Div([
        html.Div([
            html.Label("Time of Day", className='label'),
            html.Div([
                html.Button(btn['label'],
                           id={'type': 'time-btn', 'index': btn['value']},
                           n_clicks=1 if btn['value'] == 'All Day' else 0,
                           className='button active' if btn['value'] == 'All Day' else 'button')
                for btn in time_buttons
            ], className='button-group')
        ], className='card', style={'flex': '2'}),

        html.Div([
            html.Label("Month", className='label'),
            dcc.Dropdown(
                id='month-dropdown',
                options=[{'label': month, 'value': month} for month in month_options] +
                        [{'label': 'All Months', 'value': 'All Months'}],
                value='All Months',
                placeholder="Select a month",
                className='dropdown'
            )
        ], className='card', style={'flex': '1'}),

        html.Div([
            html.Label("City", className='label'),
            dcc.Dropdown(
                id='city-dropdown',
                options=[{'label': city, 'value': city} for city in city_options] +
                        [{'label': 'All Cities', 'value': 'All Cities'}],
                value='All Cities',
                placeholder="Select a city",
                className='dropdown'
            )
        ], className='card', style={'flex': '1'}),

        html.Div([
            html.Label("‎", className='label'), # Invisible char bcs I get OCD when it's not there
            html.Button("Apply Filters",
                       id='apply-filters-btn',
                       n_clicks=0,
                       className='apply-button')
        ], className='card', style={'flex': '0.8', 'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'})
    ], className='control-row'),

    # 1st row: Map Row
    html.Div([
        html.Div([
            html.H3("Accident Location Map", style={'textAlign': 'center', 'marginBottom': '15px'}),
            html.Iframe(id='leaflet-map', srcDoc=initial_map_html, width='100%', height='500')
        ], className='card', style={'width': '100%'})
    ], className='map-row', id='map-row'),

    # 2nd row: Bar Chart and Monthly Accidents
    html.Div([
        html.Div([dcc.Graph(id='vehicle-bar-chart')], className='card graph-card'),
        html.Div([dcc.Graph(id='monthly-accidents-chart')], className='card graph-card')
    ], className='graph-row', id='first-graph-row'),

    # 3rd row: Hourly Accidents
    html.Div([
        html.Div([dcc.Graph(id='hourly-accidents-chart')], className='card graph-card', style={'width': '100%'})
    ], className='graph-row', id='second-graph-row'),

    # Loading overlay (initially hidden)
    html.Div([
        html.Div([
            html.Div(className='spinner'),
            html.H3("Loading Dashboard...", style={'color': 'white', 'marginTop': '20px', 'textAlign': 'center'})
        ], className='loading-content')
    ], id='loading-overlay', className='loading-overlay', style={'display': 'none'}),

    # Store current theme state and selected time period
    dcc.Store(id='theme-store', data='light'),
    dcc.Store(id='selected-time-store', data='All Day'),

    # Footer
    html.Div([
        html.Hr(style={'margin': '40px 0 20px 0', 'border': 'none', 'height': '1px', 'opacity': '0.3'}),
        html.Div([
            # Top row - Disclaimer
            html.Div([
                html.P("DISCLAIMER: This dashboard presents information on traffic incidents in Metro Manila reported by the MMDA from 2018 to 2020.",
                       style={'margin': '0 0 20px 0', 'fontSize': '14px', 'opacity': '0.8', 'textAlign': 'center'})
            ]),
            # Bottom row - Authors
            html.Div([
                html.Div([
                    html.H4("Authors", style={'margin': '0 0 10px 0', 'fontWeight': 'bold'}),
                    html.P([
                        "Jan Robee Feliciano - ",
                        html.A("jan_feliciano@dlsu.edu.ph",
                               href="mailto:jan_feliciano@dlsu.edu.ph",
                               className="footer-link")
                    ], style={'margin': '0 0 5px 0', 'fontSize': '14px'}),
                    html.P([
                        "Alphonse Juanito Sese - ",
                        html.A("alphonse_sese@dlsu.edu.ph",
                               href="mailto:alphonse_sese@dlsu.edu.ph",
                               className="footer-link")
                    ], style={'margin': '0', 'fontSize': '14px'})
                ], style={'textAlign': 'left'}),
                html.Div([
                    html.H4("WatchMetro Dashboard", style={'margin': '0', 'fontWeight': 'bold'})
                ], style={'textAlign': 'right'})
            ], style={
                'display': 'flex',
                'justifyContent': 'space-between',
                'alignItems': 'flex-start'
            })
        ], style={
            'maxWidth': '1200px',
            'margin': '0 auto'
        })
    ], className='footer')

], id='main-container', className='light-theme')

# Callback for immediate time button visual feedback and store selection
@app.callback(
    [Output({'type': 'time-btn', 'index': ALL}, 'className'),
     Output('selected-time-store', 'data')],
    [Input({'type': 'time-btn', 'index': ALL}, 'n_clicks')],
    [State({'type': 'time-btn', 'index': ALL}, 'id')],
    prevent_initial_call=True
)
def update_time_button_styles(time_clicks, time_ids):
    ctx = callback_context
    time_filter = 'All Day'  # default

    if ctx.triggered and len(ctx.triggered) > 0:
        # Get the triggered component
        triggered_prop_id = ctx.triggered[0]['prop_id']
        if triggered_prop_id != '.':
            try:
                # Parse the component ID - more robust approach
                if 'All Day' in triggered_prop_id:
                    time_filter = 'All Day'
                elif 'Morning' in triggered_prop_id:
                    time_filter = 'Morning'
                elif 'Afternoon' in triggered_prop_id:
                    time_filter = 'Afternoon'
                elif 'Evening' in triggered_prop_id:
                    time_filter = 'Evening'
            except Exception:
                # Fallback to click count method
                time_filter = 'All Day'

    # Fallback: use the button with the highest click count
    if time_filter == 'All Day' and time_clicks:
        max_clicks = max(time_clicks) if time_clicks else 0
        if max_clicks > 0:
            max_index = time_clicks.index(max_clicks)
            time_filter = time_ids[max_index]['index']

    # Return button classes and store the selected time
    button_classes = ['button active' if id_dict['index'] == time_filter else 'button' for id_dict in time_ids]
    return button_classes, time_filter

def create_vehicle_chart(filtered_df, time_filter, selected_month, selected_city, theme_colors):
    if filtered_df.empty:
        return create_no_data_figure(
            f"Vehicles Involved in Accidents<br>({time_filter}, {selected_month}, {selected_city})",
            theme_colors,
            "No accident data available for the selected filters"
        )

    vehicles_series = filtered_df['Involved'].str.split(r"\s+AND\s+|\s*,\s*").explode()
    vehicles_series = vehicles_series.apply(clean_vehicle_name)
    vehicles_series = vehicles_series.dropna()

    if vehicles_series.empty:
        return create_no_data_figure(
            f"Vehicles Involved in Accidents<br>({time_filter}, {selected_month}, {selected_city})",
            theme_colors,
            "No accident data available for the selected filters"
        )

    vehicle_counts = vehicles_series.value_counts()

    # Filter ≥2%, merge rest into OTHERS
    percentages = vehicle_counts / vehicle_counts.sum()
    vehicle_counts_filtered = vehicle_counts[percentages >= 0.02].copy()
    others_count = vehicle_counts[percentages < 0.02].sum()
    if others_count > 0:
        vehicle_counts_filtered['OTHERS'] = others_count

    bar_chart = px.bar(
        x=vehicle_counts_filtered.values,
        y=vehicle_counts_filtered.index,
        orientation='h',
        title=f"Vehicles Involved in Accidents<br>({time_filter}, {selected_month}, {selected_city})",
        labels={'x': 'Number of Accidents', 'y': 'Vehicle Type'},
        color=vehicle_counts_filtered.index,
        color_discrete_sequence=px.colors.qualitative.G10
    )
    bar_chart.update_layout(
        plot_bgcolor=theme_colors['plot_bg'],
        paper_bgcolor=theme_colors['paper_bg'],
        font_color=theme_colors['text'],
        height=400,
        yaxis={'categoryorder': 'total ascending'}
    )
    return bar_chart

def create_monthly_chart(monthly_df, time_filter, selected_city, theme_colors):
    if monthly_df.empty:
        return create_no_data_figure(
            f"Total Accidents per Month ({time_filter}, {selected_city})",
            theme_colors,
            "No monthly data available for the selected filters"
        )

    month_counts = monthly_df['Month_Name'].value_counts().reindex(month_options, fill_value=0)

    # Check if all months have 0 accidents
    if month_counts.sum() == 0:
        return create_no_data_figure(
            f"Total Accidents per Month ({time_filter}, {selected_city})",
            theme_colors,
            "No accidents recorded in any month for the selected filters"
        )

    monthly_chart = px.bar(
        x=month_counts.index,
        y=month_counts.values,
        title=f"Total Accidents per Month ({time_filter}, {selected_city})",
        labels={'x': 'Month', 'y': 'Number of Accidents'}
    )
    monthly_chart.update_layout(
        plot_bgcolor=theme_colors['plot_bg'],
        paper_bgcolor=theme_colors['paper_bg'],
        font_color=theme_colors['text'],
        xaxis_tickangle=-45
    )
    return monthly_chart

def create_hourly_chart(filtered_df, time_filter, selected_month, selected_city, theme_colors):
    if filtered_df.empty:
        return create_no_data_figure(
            f"Accidents by Hour of Day<br>({time_filter}, {selected_month}, {selected_city})",
            theme_colors,
            "No accident data available for the selected filters"
        )

    time_range = get_time_range(time_filter)
    hourly_counts = filtered_df['Hour'].dropna().value_counts().sort_index()

    # Create hour labels in 12-hour format for the selected time range
    hour_labels = []
    hour_values = []
    for hour in time_range:
        count = hourly_counts.get(hour, 0)
        hour_values.append(count)

        # Convert to 12-hour format
        if hour == 0:
            label = "12:00 AM"
        elif hour < 12:
            label = f"{hour}:00 AM"
        elif hour == 12:
            label = "12:00 PM"
        else:
            label = f"{hour-12}:00 PM"

        hour_labels.append(label)

    # Check if all hours have 0 accidents
    if sum(hour_values) == 0:
        return create_no_data_figure(
            f"Accidents by Hour of Day<br>({time_filter}, {selected_month}, {selected_city})",
            theme_colors,
            "No accidents recorded in any hour for the selected filters"
        )

    # Use brighter blue for dark theme
    line_color = '#5DADE2' if theme_colors == dark_theme else '#2c3e50'

    hourly_chart = go.Figure()
    hourly_chart.add_trace(go.Scatter(
        x=hour_labels,
        y=hour_values,
        mode='lines+markers',
        name='Accidents',
        line=dict(color=line_color, width=2),
        marker=dict(size=6)
    ))

    hourly_chart.update_layout(
        title=f"Accidents by Hour of Day<br>({time_filter}, {selected_month}, {selected_city})",
        xaxis_title="Hour of Day",
        yaxis_title="Number of Accidents",
        plot_bgcolor=theme_colors['plot_bg'],
        paper_bgcolor=theme_colors['paper_bg'],
        font_color=theme_colors['text'],
        xaxis=dict(tickangle=-45)
    )
    return hourly_chart

# Main callback for theme toggle (immediate)
@app.callback(
    [Output('main-container', 'className'),
     Output('theme-store', 'data')],
    [Input('theme-toggle', 'value')]
)
def update_theme(theme_value):
    theme = 'dark' if 'dark' in theme_value else 'light'
    return f'{theme}-theme', theme

@app.callback(
    [Output('vehicle-bar-chart', 'figure'),
     Output('monthly-accidents-chart', 'figure'),
     Output('hourly-accidents-chart', 'figure'),
     Output('loading-overlay', 'style')],
    [Input('theme-store', 'data'),
     Input('apply-filters-btn', 'n_clicks')],
    [State('selected-time-store', 'data'),
     State('month-dropdown', 'value'),
     State('city-dropdown', 'value')],
    prevent_initial_call=False
)
def update_graphs_and_loading(theme, apply_clicks, time_filter, selected_month, selected_city):
    ctx = callback_context

    # Determine if this was triggered by apply button
    is_apply_button = ctx.triggered and 'apply-filters-btn' in ctx.triggered[0]['prop_id']

    theme_colors = dark_theme if theme == 'dark' else light_theme

    # Use current filter values or defaults
    if not time_filter:
        time_filter = 'All Day'
    if not selected_month:
        selected_month = 'All Months'
    if not selected_city:
        selected_city = 'All Cities'

    # Filter data
    filtered_df = df.copy()

    # Apply month filter
    if selected_month != 'All Months':
        filtered_df = filtered_df[filtered_df['Month_Name'] == selected_month]

    # Apply city filter
    if selected_city != 'All Cities':
        filtered_df = filtered_df[filtered_df['City'] == selected_city]

    # Apply time filter
    if time_filter != 'All Day':
        filtered_df = filtered_df[filtered_df['Time_Period'] == time_filter]

    # Create charts using enhanced functions
    bar_chart = create_vehicle_chart(filtered_df, time_filter, selected_month, selected_city, theme_colors)

    # For monthly chart, use different filtering
    monthly_df = df.copy()
    if time_filter != 'All Day':
        monthly_df = monthly_df[monthly_df['Time_Period'] == time_filter]
    if selected_city != 'All Cities':
        monthly_df = monthly_df[monthly_df['City'] == selected_city]
    monthly_chart = create_monthly_chart(monthly_df, time_filter, selected_city, theme_colors)

    hourly_chart = create_hourly_chart(filtered_df, time_filter, selected_month, selected_city, theme_colors)

    # Hide loading overlay after processing
    loading_style = {'display': 'none'}

    return bar_chart, monthly_chart, hourly_chart, loading_style

# Separate callback for showing loading overlay immediately when apply button is clicked
@app.callback(
    Output('loading-overlay', 'style', allow_duplicate=True),
    [Input('apply-filters-btn', 'n_clicks')],
    prevent_initial_call=True
)
def show_loading_immediately(n_clicks):
    if n_clicks > 0:
        return {'display': 'flex'}
    return {'display': 'none'}

# Separate callback for updating map only when apply filters is clicked
@app.callback(
    Output('leaflet-map', 'srcDoc'),
    [Input('apply-filters-btn', 'n_clicks')],
    [State('selected-time-store', 'data'),
     State('month-dropdown', 'value'),
     State('city-dropdown', 'value'),
     State('theme-store', 'data')],
    prevent_initial_call=True
)
def update_map_only(apply_clicks, time_filter, selected_month, selected_city, theme):
    # Use stored time filter or default
    if not time_filter:
        time_filter = 'All Day'
    if not selected_month:
        selected_month = 'All Months'
    if not selected_city:
        selected_city = 'All Cities'

    # Filter data
    filtered_df = df.copy()

    # Apply month filter
    if selected_month != 'All Months':
        filtered_df = filtered_df[filtered_df['Month_Name'] == selected_month]

    # Apply city filter
    if selected_city != 'All Cities':
        filtered_df = filtered_df[filtered_df['City'] == selected_city]

    # Apply time filter
    if time_filter != 'All Day':
        filtered_df = filtered_df[filtered_df['Time_Period'] == time_filter]

    # Update Map (always use light theme for now)
    map_data = filtered_df.dropna(subset=['Latitude', 'Longitude'])
    updated_map = generate_folium_map(map_data, 'light')

    try:
        buffer = BytesIO()
        updated_map.save(buffer, close_file=False)
        map_html = buffer.getvalue().decode("utf-8")
        buffer.close()
        return map_html
    except Exception as e:
        # Fallback to initial map if there's an error
        return initial_map_html

# Custom CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>WatchMetro: Metro Manila Accidents Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            body { margin: 0; font-family: 'Arial', sans-serif; }

            .header-container {
                display: flex;
                justify-content: space-between;
                align-items: center;
                padding: 20px 40px;
                margin-bottom: 20px;
            }

            .theme-toggle-container {
                display: flex;
                align-items: center;
                gap: 8px;
            }

            .theme-icon {
                font-size: 20px;
                opacity: 0.7;
                transition: opacity 0.3s ease;
            }

            .toggle-checklist {
                margin: 0 !important;
            }

            .toggle-checklist label {
                background-color: #ccc;
                border-radius: 24px;
                transition: 0.4s;
                position: relative;
                cursor: pointer;
                margin: 0 !important;
                display: block;
                width: 50px;
                height: 24px;
            }

            .toggle-checklist label:before {
                position: absolute;
                content: "";
                height: 18px;
                width: 18px;
                left: 3px;
                top: 3px;
                background-color: white;
                transition: 0.4s;
                border-radius: 50%;
            }

            /* Fixed toggle slider - using more specific selectors */
            .toggle-checklist input[type="checkbox"]:checked + label {
                background-color: #2c3e50 !important;
            }

            .toggle-checklist input[type="checkbox"]:checked + label:before {
                transform: translateX(26px) !important;
            }

            /* Alternative approach using has() selector for modern browsers */
            .toggle-checklist label:has(input:checked) {
                background-color: #2c3e50 !important;
            }

            .toggle-checklist label:has(input:checked):before {
                transform: translateX(26px) !important;
            }

            .apply-button {
                padding: 12px 20px;
                background: linear-gradient(135deg, #3498db, #2980b9);
                color: white;
                border: none;
                border-radius: 8px;
                cursor: pointer;
                font-weight: 600;
                font-size: 14px;
                transition: all 0.3s ease;
                width: 100%;
                height: auto;
            }

            .apply-button:hover {
                background: linear-gradient(135deg, #2980b9, #2471a3);
                transform: translateY(-2px);
                box-shadow: 0 4px 12px rgba(52, 152, 219, 0.4);
            }

            .loading-overlay {
                position: fixed;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                background-color: rgba(0, 0, 0, 0.8);
                z-index: 9999;
                display: flex;
                justify-content: center;
                align-items: center;
                flex-direction: column;
            }

            .loading-content {
                text-align: center;
            }

            .spinner {
                border: 4px solid #f3f3f3;
                border-top: 4px solid #3498db;
                border-radius: 50%;
                width: 50px;
                height: 50px;
                animation: spin 1s linear infinite;
                margin: 0 auto;
            }

            @keyframes spin {
                0% { transform: rotate(0deg); }
                100% { transform: rotate(360deg); }
            }

            .control-row {
                display: flex;
                flex-wrap: wrap;
                justify-content: center;
                gap: 20px;
                margin-bottom: 30px;
                padding: 10px;
            }

            .graph-row, .map-row {
                display: flex;
                flex-wrap: wrap;
                gap: 20px;
                justify-content: center;
                margin-bottom: 30px;
                padding: 10px;
            }

            .card {
                border-radius: 12px;
                box-shadow: 0 4px 12px rgba(0,0,0,0.1);
                padding: 20px;
                transition: all 0.3s ease;
                background-color: white;
                color: black;
            }

            .dark-theme .card {
                background-color: #2c2c2c;
                color: white;
                box-shadow: 0 4px 12px rgba(0,0,0,0.3);
            }

            .label {
                font-weight: bold;
                margin-bottom: 10px;
                display: block;
                font-size: 14px;
            }

            /* Fixed: dropdown labels turn white in dark mode */
            .dark-theme .label {
                color: white !important;
            }

            .button-group {
                display: flex;
                gap: 8px;
                flex-wrap: wrap;
            }

            .button {
                padding: 10px 16px;
                background-color: #e0e0e0;
                border: none;
                border-radius: 8px;
                cursor: pointer;
                transition: all 0.3s ease;
                font-weight: 500;
                font-size: 14px;
            }

            .button:hover {
                background-color: #ccc;
                transform: translateY(-2px);
                box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            }

            .active {
                background-color: #2c3e50 !important;
                color: white !important;
                box-shadow: 0 2px 8px rgba(44,62,80,0.3);
            }

            .graph-card {
                flex: 1 1 500px;
                min-height: 400px;
            }

            .dropdown {
                width: 100%;
            }

            .light-theme {
                background-color: #f9f9f9;
                color: #111111;
                min-height: 100vh;
            }

            .dark-theme {
                background-color: #1e1e1e;
                color: white;
                min-height: 100vh;
            }

            .title {
                font-size: 2.5em;
                font-weight: bold;
                background: linear-gradient(45deg, #2c3e50, #3498db);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                background-clip: text;
            }

            .dark-theme .title {
                background: linear-gradient(45deg, #ecf0f1, #3498db);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                background-clip: text;
            }

            /* Enhanced dropdown styling for dark theme */
            .dark-theme .Select-control,
            .dark-theme .Select-single-value,
            .dark-theme .Select-input {
                background-color: #2c2c2c !important;
                border-color: #555 !important;
                color: white !important;
            }

            .dark-theme .Select-menu-outer {
                background-color: #2c2c2c !important;
                border-color: #555 !important;
            }

            .dark-theme .Select-option {
                background-color: #2c2c2c !important;
                color: white !important;
            }

            .dark-theme .Select-option:hover {
                background-color: #444 !important;
            }

            .dark-theme .Select-arrow {
                border-color: white transparent transparent transparent !important;
            }

            .dark-theme.is-open > .Select-control .Select-arrow {
                border-color: transparent transparent white transparent !important;
            }

            /* Additional dropdown styling for better dark theme support */
            .dark-theme .dash-dropdown .Select-control {
                background-color: #2c2c2c !important;
                border: 1px solid #555 !important;
                color: white !important;
            }

            .dark-theme .dash-dropdown .Select-value-label,
            .dark-theme .dash-dropdown .Select-placeholder {
                color: white !important;
            }

            .dark-theme .dash-dropdown .Select-menu {
                background-color: #2c2c2c !important;
                border: 1px solid #555 !important;
            }

            .dark-theme .dash-dropdown .VirtualizedSelectOption {
                background-color: #2c2c2c !important;
                color: white !important;
            }

            .dark-theme .dash-dropdown .VirtualizedSelectFocusedOption {
                background-color: #444 !important;
                color: white !important;
            }

            /* Footer Styles */
            .footer {
                background-color: transparent;
                padding: 20px 40px 30px 40px;
                margin-top: 50px;
                border-top: 1px solid rgba(0,0,0,0.1);
            }

            .dark-theme .footer {
                border-top: 1px solid rgba(255,255,255,0.1);
            }

            .footer-link {
                color: #3498db;
                text-decoration: none;
                transition: color 0.3s ease;
            }

            .footer-link:hover {
                color: #2980b9;
                text-decoration: underline;
            }

            .dark-theme .footer-link {
                color: #5DADE2;
            }

            .dark-theme .footer-link:hover {
                color: #85C1E9;
            }

            @media (max-width: 768px) {
                .footer > div > div:last-child > div {
                    flex-direction: column !important;
                    text-align: center !important;
                    gap: 20px;
                }

                .footer > div > div:last-child > div > div {
                    text-align: center !important;
                }
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

if __name__ == '__main__':
    app.run(debug=False)


<IPython.core.display.Javascript object>